1. Select best optimizer
2. Select number of neurons in a layer
3. Select number of layers
4. All in one model.

In [1]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [2]:
import kerastuner as kt

import pandas as pd
import numpy as np

import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

/tmp/ipython-input-1039753031.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/pima-indians-diabetes-database")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'pima-indians-diabetes-database' dataset.
Path to dataset files: /kaggle/input/pima-indians-diabetes-database


In [4]:
# concatenate path with the file name

path  = path + '/diabetes.csv'

In [5]:
df = pd.read_csv(path)

In [7]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [8]:
x = df.iloc[:,0:-1]
y=df.iloc[:,-1]

In [9]:
standardScalar = StandardScaler()

x = standardScalar.fit_transform(x)

In [10]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2, random_state = 1)

#1. Select best optimizer

1.1 We will first build a model using RMSProp optimizer.
   
    Using RMSProp val_accuracy: 0.7922.

2.1 Using Keras tuner we will find out best optimizer first
   
   it is coming out to be Adam.
   
3.1 Using this model train the model
   
   val_accuracy is uptated to 0.82





In [11]:
model = Sequential()
model.add(Dense(32,activation='relu', input_dim = 8))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(xtrain,ytrain,epochs=100,batch_size = 32, validation_split = 0.2, validation_data=(xtest,ytest))
model.summary()

Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5318 - loss: 0.7611 - val_accuracy: 0.5649 - val_loss: 0.6946
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6321 - loss: 0.6610 - val_accuracy: 0.6364 - val_loss: 0.6373
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6897 - loss: 0.6179 - val_accuracy: 0.7013 - val_loss: 0.5934
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6914 - loss: 0.5799 - val_accuracy: 0.7338 - val_loss: 0.5652
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7168 - loss: 0.5694 - val_accuracy: 0.7597 - val_loss: 0.5450
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7181 - loss: 0.5548 - val_accuracy: 0.7662 - val_loss: 0.5265
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7665 - loss: 0.5155 - val_accuracy: 0.7727 - val_loss: 0.5145
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7466 - loss: 0.5172 - val_accuracy: 0.7727 - val_loss: 0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 644 (2.52 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 323 (1.27 KB)

#1. Select best optimizer

In [12]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(32,activation='relu', input_dim = 8))
  model.add(Dense(1, activation='sigmoid'))
  optimizer_list = hp.Choice('optimizer', values=['adam','sgd','rmsprop', 'adadelta'])
  model.compile(optimizer=optimizer_list, loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [13]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

In [14]:
tuner.search(xtrain,ytrain,epochs=5,validation_data=(xtest,ytest))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.5259740352630615

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 12s


In [15]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [16]:
model_optimizer = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [17]:
model_optimizer.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# initial epoch is set as 6 since till 5 it is already fitted above.
model_optimizer.fit(xtrain,ytrain,batch_size=32,epochs=100,initial_epoch=6,validation_data=(xtest,ytest))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7555 - loss: 0.5256 - val_accuracy: 0.7727 - val_loss: 0.5041
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7705 - loss: 0.5106 - val_accuracy: 0.7662 - val_loss: 0.4930
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7791 - loss: 0.5035 - val_accuracy: 0.7727 - val_loss: 0.4849
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7907 - loss: 0.4611 - val_accuracy: 0.7727 - val_loss: 0.4794
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7577 - loss: 0.4916 - val_accuracy: 0.7727 - val_loss: 0.4762
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8046 - loss: 0.4364 - val_accuracy: 0.7727 - val_loss: 0.4736
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7942 - loss: 0.4586 - val_accuracy: 0.7792 - val_loss: 0.4723
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7839 - loss: 0.4510 - val_accuracy: 0.77

# 2. Select number of neurons in a layer

2.1 instead of directly providing number of neurons, we can use the list and provide min and max numbers and step size.

it comes out to be 80 neurons per layer

In [19]:
def build_model(hp):
  model=Sequential()
  units_list = hp.Int('units',min_value=8,max_value = 128,step=1)
  model.add(Dense(units=units_list, activation='relu', input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
  return model

In [20]:
tuner_units = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='ex_unit_dir', project_name='ex_unit_project')

In [21]:
tuner_units.search(xtrain,ytrain,epochs=5, validation_data=(xtest,ytest))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 16s


In [22]:
tuner_units.get_best_hyperparameters()[0].values

{'units': 45}

In [23]:
model_best_units = tuner_units.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [24]:
model_best_units.fit(xtrain,ytrain,batch_size=32,epochs=100,initial_epoch=6, validation_data=(xtest,ytest))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7333 - loss: 0.5246 - val_accuracy: 0.8182 - val_loss: 0.4905
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7656 - loss: 0.4963 - val_accuracy: 0.8182 - val_loss: 0.4804
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7667 - loss: 0.4906 - val_accuracy: 0.8117 - val_loss: 0.4740
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7751 - loss: 0.4756 - val_accuracy: 0.8182 - val_loss: 0.4700
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7730 - loss: 0.4829 - val_accuracy: 0.8182 - val_loss: 0.4653
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7832 - loss: 0.4666 - val_accuracy: 0.8247 - val_loss: 0.4636
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7809 - loss: 0.4744 - val_accuracy: 0.8247 - val_loss: 0.4630
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7761 - loss: 0.4810 - val_accuracy: 0.81

#3. Select number of layers

In [29]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(80, activation='relu', input_dim=8))

  for i in range(hp.Int('num_layers', min_value=1,max_value=10)):
    model.add(Dense(80,activation=hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])))

  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
  return model

In [30]:
tuner_layers = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='ex_layers_dir', project_name='ex_layers_project')

In [31]:
tuner_layers.search(xtrain,ytrain,epochs=5, validation_data=(xtest,ytest))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.8246753215789795

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 23s


In [32]:
tuner_layers.get_best_hyperparameters()[0].values

{'num_layers': 4, 'activation': 'tanh'}

In [33]:
model_best_layers = tuner_layers.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [34]:
model_best_layers.fit(xtrain,ytrain,batch_size=32,epochs=100,initial_epoch=6, validation_data=(xtest,ytest))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.7290 - loss: 0.5066 - val_accuracy: 0.7792 - val_loss: 0.4778
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8067 - loss: 0.4423 - val_accuracy: 0.7987 - val_loss: 0.4741
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7586 - loss: 0.4773 - val_accuracy: 0.8182 - val_loss: 0.4720
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7954 - loss: 0.4550 - val_accuracy: 0.7792 - val_loss: 0.4685
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7862 - loss: 0.4243 - val_accuracy: 0.7987 - val_loss: 0.4515
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7821 - loss: 0.4690 - val_accuracy: 0.7922 - val_loss: 0.4712
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7755 - loss: 0.4511 - val_accuracy: 0.7987 - val_loss: 0.4544
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7732 - loss: 0.4391 - val_accuracy: 0.77

# Lets make another version with all suggest changes

In [35]:
tuner_layers.search(xtrain,ytrain,epochs=5, validation_data=(xtest,ytest))